In [18]:
pip install pyquery

Note: you may need to restart the kernel to use updated packages.


In [19]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd
import re

In [20]:
def get_information(since_id = '',uid = 0,):
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}
    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '100103type=1&q=?',
                'type': int(uid),
                'value': 5448226524,
                'containerid': '107603' + str(uid),
                'since_id': since_id
    }
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    json_text = response.json()
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [21]:
def get_fulltext(mid = ''):
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}
    params = {
                'id':mid
    }
    url = 'https://m.weibo.cn/statuses/extend?' + urlencode(params)
    response = requests.get(url,headers = headers)
    json_text = response.json()
    print('---',json_text)
    if json_text.get('ok')==0:
        return ''
    else:
        text = re.sub("<[^>]*?>","",json_text.get('data').get('longTextContent'))
        return text

In [22]:
def parse_json(json):
    items = json.get('data').get('cards')
    for item in items:
        item = item.get('mblog')
        mid = item.get('id')
        weibo = {}
        weibo['Time'] = item.get('created_at')
        weibo['Text'] = get_fulltext(mid)
        yield weibo

In [44]:
uid = 3186551317
p = 50
s = get_information(since_id = '',uid = uid)

data = []
for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    for i in parse_json(s[0]):
        print(i)
        data.append(i)
pd.DataFrame(data)
# pd.DataFrame(data).to_csv('data22.csv')
# pd.DataFrame(data).to_csv('/Users/snow/Desktop/data22.csv')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
pd.DataFrame(data).to_csv('健康广州.csv', encoding='utf_8_sig')